In [5]:
### Inputs: traces_dict, node_details_dict and trace_details_dict
# Node details dict= nid: [nis, type]
### Config file: DB split and SLtype split
### Outputs: updated_node_details

In [10]:
import pickle
import yaml
import random
import json

import networkx as nx
import numpy as np
from collections import Counter

In [11]:
def pkl_to_dict(file_path):
    with open(file_path, 'rb') as pkl_file:
        T_prime = pickle.load(pkl_file)
    return T_prime

def save_dict_as_pkl(traces_dict, file_name):
    with open(file_name+'.pkl', 'wb') as pkl_file:
        pickle.dump(traces_dict, pkl_file)

def save_dict_as_json(traces_dict, file_name):
    with open(file_name+'.json', 'w') as json_file:
        json.dump(traces_dict, json_file)

def read_yaml(file):
    with open(file, 'r') as f:
        data = yaml.safe_load(f)
    return data

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print(f"The file '{file_path}' was not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file '{file_path}'.")

def build_digraph_from_tracesdict(traces_dict):

    full_graph_edge_list = []
    for edge_list in traces_dict.values():
        full_graph_edge_list.extend(edge_list)
    G = nx.DiGraph()
    G.add_edges_from(full_graph_edge_list)

    return G

def prune_node_details(traces_dict, node_dets):
    nodes_from_traces = []
    for _, e_list in traces_dict.items():
        for e in e_list:
            if e[0] not in nodes_from_traces:
                nodes_from_traces.append(e[0])
            if e[1] not in nodes_from_traces:
                nodes_from_traces.append(e[1])
    pruned_node_dets = {node: details for node, details\
                         in node_dets.items() if node in nodes_from_traces}

    return pruned_node_dets

def calc_graph_depth(G, initial_node):
    def dfs(node, visited, stack):
        # Using stack to avoid cycles
        if node in stack:
            return 0
        if node in visited:
            return visited[node]
        stack.add(node)
        max_depth = 0
        for neighbour in G.successors(node):
            depth = dfs(neighbour, visited, stack)
            max_depth = max(max_depth, depth)
        stack.remove(node)
        visited[node] = max_depth + 1
        
        return visited[node]
    
    visited = {}
    stack = set()

    return dfs(initial_node, visited, stack)
def find_inode_and_graph_depth(G):
    max_depth = -1
    node_with_max_depth = ''

    for node in G.nodes():
        depth = calc_graph_depth(G, node)
        if depth > max_depth:
            max_depth = depth
            node_with_max_depth = node
    return node_with_max_depth, max_depth

In [12]:
# Read in configs
config = read_yaml('enrichment_config.yaml')
databases = config['Databases']
workload_name = config['ExpWorkloadName']
## Datastore mixtures: dmix1_pg_heavy, dmix2_mongo_heavy, dmix3_redis_heavy
## Consistency exp: cons_exp (Mongo:Redis:Postgres; 40:40:20)
# workload_name = "cons_exp"

'''
NODE ENRICHMENT
---------------
Input: Traces_dict, Node_details_dict
Output: Node split output
        node_split_output = {'sf_split': {DB1: {'count': 30, 'nodes_list': [nid1, nid2, ...]}, ...},..}
                             'sl_split': ,,}
'''

traces_dict = pkl_to_dict('traces/final_500nodes_250ktraces.pkl')
node_dets = pkl_to_dict('node_and_trace_details/final_node_details_data.pkl') # Node details dict= nid: [nis, SF, DB_name] (or) [nis, SL, SL_type]

node_dets = prune_node_details(traces_dict, node_dets)
print("Num nodes in node_dets: ", len(node_dets))

sf_arr = [nid for nid, n_info in node_dets.items() if n_info[1] == "db"]
sl_arr = [nid for nid, n_info in node_dets.items() if n_info[1] != "db"]

sf_count = len(sf_arr)
print("Number of SF nodes in trace graph:", sf_count)
sl_count = len(sl_arr)
print("Number of SL nodes in trace graph:", sl_count)
total_nodes = sf_count + sl_count

db_split_arr = [[db_name, info['percentage']] for db_name, info in databases.items()]# [[DB1, 30%],...]
sl_type_split = [['Python', sl_count]]
print("Database split Input:", db_split_arr)

def percent_to_count(arr, count):
    raw_counts = [round(count * (i[1] / 100)) for i in arr]
    diff = count - sum(raw_counts)
    
    # Distribute the difference
    idx = 0
    while diff != 0:
        if diff > 0:
            raw_counts[idx] += 1  # Increase by 1 if we need to add
            diff -= 1
        elif diff < 0:
            raw_counts[idx] -= 1  # Decrease by 1 if we need to remove
            diff += 1
        idx = (idx + 1) % len(raw_counts)
    
    for idx, i in enumerate(arr):
        arr[idx] = [i[0], raw_counts[idx]]
    
    return arr

db_split_arr = percent_to_count(db_split_arr, sf_count) # nid: [nis, SF, DB_name]
print("Database split output:", db_split_arr)


sf_split_info = {ntype: {"count": value, "nodes_list": []} for ntype, value in db_split_arr}
sl_split_info = {'Python': {"count": sl_count, "nodes_list": []}}

def assign_nodes_to_types(split_arr, sfsl_arr, split_info):
    sfsl_arr_cpy = sfsl_arr.copy()
    # Assign nodes to db and sl types
    for i in split_arr:
        ctr = 0
        name = i[0] # type name: eg: Mongo, Redis, Relay
        for _ in range(i[1]):
            ctr += 1
            nid = sfsl_arr_cpy.pop(random.randint(0, len(sfsl_arr_cpy) - 1))
            node_dets[nid].append(name) # add type to node details
            split_info[name]["nodes_list"].append(nid) # add node to list of nodes for that type
        # print(ctr, name)
    return node_dets, split_info

node_dets, sf_split_info = assign_nodes_to_types(db_split_arr, sf_arr, sf_split_info)
node_dets, sl_split_info = assign_nodes_to_types(sl_type_split, sl_arr, sl_split_info)

# Saving node split output
node_split_output = {'sf_split': sf_split_info, 'sl_split': sl_split_info}
print("Nodes Split Output:", node_split_output)
save_dict_as_json(node_split_output, f'enrichment_runs/{workload_name}/node_split_output')
# save_dict_as_json(node_split_output, f'enrichment_runs/as_dmix1_2/node_split_output') ################# AS EXP

Num nodes in node_dets:  498
Number of SF nodes in trace graph: 247
Number of SL nodes in trace graph: 251
Database split Input: [['MongoDB', 15], ['Redis', 15], ['Postgres', 70]]
Database split output: [['MongoDB', 37], ['Redis', 37], ['Postgres', 173]]
Nodes Split Output: {'sf_split': {'MongoDB': {'count': 37, 'nodes_list': ['n1634', 'n141', 'n5406', 'n1242', 'n1253', 'n2267', 'n6396', 'n9619', 'n6167', 'n4918', 'n2160', 'n1871', 'n8838', 'n4914', 'n5451', 'n5278', 'n8589', 'n8189', 'n1925', 'n2617', 'n1553', 'n41', 'n2016', 'n6799', 'n1962', 'n2292', 'n7244', 'n2812', 'n4835', 'n2375', 'n4191', 'n4034', 'n9367', 'n7885', 'n6442', 'n7222', 'n483']}, 'Redis': {'count': 37, 'nodes_list': ['n2143', 'n3462', 'n7768', 'n7139', 'n4534', 'n4987', 'n6278', 'n3594', 'n4133', 'n3931', 'n6006', 'n8408', 'n3578', 'n4815', 'n5441', 'n4271', 'n1514', 'n7186', 'n8405', 'n7025', 'n1935', 'n4498', 'n1232', 'n7765', 'n7681', 'n2357', 'n3444', 'n8699', 'n2184', 'n5790', 'n2479', 'n3526', 'n4797', 'n893

In [ ]:
'''
Object id Enrichment
Output: Trace packets.
        Trace packets = [t_node_calls_dict, t_data_ops_dict]
        t_node_calls_dict = Key: dm node, Value: list of [dm node, op_id]
        t_data_ops_dict = Key: data op id, Value: data op packet
'''

class Wl_config:
    """
    Format: record_count, record_size_dist,
                 data_access_pattern, rw_ratio, async_sync_ratio, seed
    """
    def __init__(self, record_count, record_size_dist,\
                 data_access_pattern, rw_ratio, async_sync_ratio, seed):
        self.record_count = record_count
        self.record_size_dist = record_size_dist
        self.data_access_pattern = data_access_pattern
        self.rw_ratio = rw_ratio
        self.async_sync_ratio = async_sync_ratio
        self.seed = seed

        # Setting seed
        np.random.seed(self.seed)
        random.seed(self.seed)
        # Generate object sizes and data access pattern
        self.obj_ids_list = np.arange(1, self.record_count + 1)
        self.object_sizes_dict = self.generate_object_sizes()
        self.probabilities = self.generate_data_access_pattern()

    def generate_object_sizes(self):
        if self.record_size_dist == 'lognormal':
            obj_sizes = np.random.lognormal(mean=np.log(self.record_count), \
                                                 sigma=np.log(self.record_count), \
                                                 size=self.record_count)
        elif self.record_size_dist == 'uniform':
            obj_sizes = np.random.uniform(low=1, high=self.record_count, size=self.record_count)
        else:
            raise ValueError('Invalid record size distribution, only lognormal & uniform are allowed for now')
        return dict(zip(self.obj_ids_list, obj_sizes))
    
    def generate_data_access_pattern(self):
        if self.data_access_pattern == 'zipfian':
            alpha = 1.2
            probabilities = np.random.zipf(alpha, len(self.obj_ids_list))
            probabilities = probabilities / probabilities.sum()
        elif self.data_access_pattern == 'uniform':
            probabilities = np.ones(len(self.obj_ids_list)) / len(self.obj_ids_list)
        else:
            raise ValueError('Invalid data access pattern, only zipfian & uniform are allowed for now.')
        return probabilities


def gen_sfnode_dataops(sf_node, wl_config, traces_dict, node_dets, node_split_output):
    '''
    For a given sf node, generate data ops (count total dm calls to sf node)
    Return: ops_dict= Key: op_id, Value: op_packet
    op_packet = {'op_id': op_id, 'op_type': op_type, 'op_obj_id': op_obj_id,\
                 'db': sf_node_db}
    '''
    obj_ids_list = wl_config.obj_ids_list
    # obj_sizes_dict = wl_config.object_sizes_dict
    data_acc_probabilities = wl_config.probabilities
    w_prob = wl_config.rw_ratio / (1 + wl_config.rw_ratio)

    # sf_node_db = node_dets[sf_node][2]
    sf_node_db = None
    for db_type, db_info in node_split_output['sf_split'].items():
        if sf_node in db_info['nodes_list']:
            sf_node_db = db_type
            break
    if sf_node_db is None:
        raise ValueError(f"Could not determine DB type for node: {sf_node}")

   # find the number of ops to be generated
    total_ops = 0
    for e_list in traces_dict.values():# count total dm calls to sf node
        for e in e_list:
            if e[1] == node:
                total_ops += 1

    # generate ops for sf node
    written_obj_ids = set()  
    ops_dict = {}   # key: op_id, value: op_packet
    for op_id in range(1, total_ops + 1):
        # op_type = 'write' if random.random() < w_prob else 'read'
        # op_obj_id = np.random.choice(obj_ids_list,\
        #                              p=data_acc_probabilities)# Select by data access pattern
        op_obj_id = random.randrange(1, wl_config.record_count + 1)
        if op_obj_id not in written_obj_ids:
            op_type = 'write'
            written_obj_ids.add(op_obj_id)
        else:
            op_type = 'write' if random.random() < w_prob else 'read'

        # op_obj_size = obj_sizes_dict[op_obj_id]
        operation = {'op_id': op_id, 'op_type': op_type, 'op_obj_id': f"key_{op_obj_id}",\
                      'db': sf_node_db} # op_packet
        ops_dict[op_id] = operation
    
    return ops_dict


# convert edges_list to node_calls_dict format 
def gen_node_calls_dict(edges_list, async_sync_ratio):
    '''
    Return: node_calls_dict = Key: dm node, Value: list of [dm node, op_id, async_flag]
            (op_id = -1 for SL) (async_flag = 1 for async, 0 for sync)
    '''
    node_calls_dict = {}
    for edge in edges_list:
        if edge[0] not in node_calls_dict:
            node_calls_dict[edge[0]] = []
        async_prob = async_sync_ratio / (1 + async_sync_ratio)
        async_flag = 1 if random.random() < async_prob else 0
        node_calls_dict[edge[0]].append([edge[1], -1, async_flag]) # [dm node, op_id, async/sync] (-1 for SL) (1/0 for async/sync)
    return node_calls_dict

def get_pop_first_dict_item(d):
    first_key = list(d.keys())[0]
    first_item = d.pop(first_key)
    return first_key, first_item

def get_node_type(node_id, data):
    '''
    data: node_split_output.json
    '''
    for split_type, services in data.items():
        for service, service_data in services.items():
            if node_id in service_data['nodes_list']:
                return service

def remove_self_node_calls(node_call_dict):
    for node, dm_nodes in node_call_dict.items():
        node_call_dict[node] = [dm_node for dm_node in dm_nodes if dm_node[0] != node]
    return node_call_dict

def get_leaf_nodes(node_call_dict):
    '''Returns: leaf nodes in a request call graph'''
    all_nodes = set(node_call_dict.keys())
    called_nodes = set()
    for calls in node_call_dict.values():
        for call in calls:
            called_nodes.add(call[0])
    leaf_nodes = called_nodes - all_nodes
    return leaf_nodes

def get_logger_nodes_for_request_call_graph(node_call_dict):
    '''Returns: list of nodes that log for the request call graph
                SL leaf nodes and SL node predecessor to SF leaf nodes.
    '''
    logger_nodes = set()
    t_leaf_nodes = get_leaf_nodes(node_call_dict) # find all leaf nodes
    for ln in t_leaf_nodes:
        for node, calls in node_call_dict.items():
            for call in calls:
                if call[0] == ln and call[1] != -1: # Leaf SF node
                    logger_nodes.add(node)
                elif call[0] == ln and call[1] == -1: # Leaf SL node
                    logger_nodes.add(ln)
    return list(logger_nodes)

def has_cycle(graph):
    def dfs(node, visited, rec_stack):
        if node not in visited:
            # Mark the current node as visited and add to the recursion stack
            visited.add(node)
            rec_stack.add(node)
            # Check all the nodes this node is connected to
            for neighbor_info in graph.get(node, []):
                neighbor = neighbor_info[0]
                # If the neighbor is not visited, do a recursive DFS call
                if neighbor not in visited and dfs(neighbor, visited, rec_stack):
                    return True
                # If the neighbor is already in the recursion stack, it's a cycle
                elif neighbor in rec_stack:
                    return True
            rec_stack.remove(node)
        return False
    visited = set()
    rec_stack = set()
    # Check for cycles starting from each node in the graph
    for node in graph.keys():
        if node not in visited:
            if dfs(node, visited, rec_stack):
                return True
    return False

# Reading enrichment config file
enrichment_config = read_yaml('enrichment_config.yaml')
record_count = enrichment_config['WorkloadConfig']['record_count']
record_size_dist = enrichment_config['WorkloadConfig']['record_size_dist']
data_access_pattern = enrichment_config['WorkloadConfig']['data_access_pattern']
rw_ratio = enrichment_config['WorkloadConfig']['rw_ratio']
async_sync_ratio = enrichment_config['WorkloadConfig']['async_sync_ratio']
# Format: record_count, record_size_dist, data_access_pattern, rw_ratio, async_sync_ratio, seed
wl1 = Wl_config(record_count, record_size_dist, data_access_pattern, rw_ratio, async_sync_ratio, seed=50) # to be read from config file
node_split_output = json.load(open(f'./enrichment_runs/{workload_name}/node_split_output.json'))

'''
Generate data op packets for each sf node
Returns: overall_data_ops = key: sf_node, value: ops_dict {ops_id: op_packet}
'''
G_agg = build_digraph_from_tracesdict(traces_dict)
overall_data_ops = {}   # key: sf_node, value: ops_dict {ops_id: op_packet}
check = 0
for node in node_dets:
    if node in G_agg.nodes() and node_dets[node][1] == 'db':
        overall_data_ops[node] = \
            gen_sfnode_dataops(node, wl1, traces_dict, node_dets, node_split_output)



'''
Making the trace packet:
trace_packet = [t_node_calls_dict, t_data_ops_dict, t_ini_node, t_ini_node_type]
t_node_calls_dict = Key: dm node, Value: list of [dm node, op_id]
t_data_ops_dict = Key: data op id, Value: data op packet
'''
def add_if_not_present(array, value):
    if value not in array:
        array.append(value)
    return array
# trace_details_data = pkl_to_dict('node_and_trace_details/new_trace_details_data.pkl')

# node_split_output = json.load(open(f'./enrichment_runs/as_dmix1_2/node_split_output.json')) ################# AS EXP
all_trace_packets = {}
cycle_ctr = 0
unique_nodes_check = []
for tid in traces_dict:
    t_node_calls_dict = gen_node_calls_dict(traces_dict[tid], async_sync_ratio)
    t_data_ops_dict = {} # key: data op id, value: data op packet
    for t_node in t_node_calls_dict:
        for idx in range(len(t_node_calls_dict[t_node])):# Why is it not entering the if loop?
            dm_node = t_node_calls_dict[t_node][idx][0]
            if node_dets[dm_node][1] == 'db': # ie if dm node is a sf node
                # Select a data op id from the data ops dict and pop it
                if len(overall_data_ops[dm_node]) == 0:
                    print("Error: No data ops for sf node", dm_node)
                    break
                # Select a data op id from the data ops dict and pop it
                op_id, op_packet = get_pop_first_dict_item(overall_data_ops[dm_node])
                if node_dets[dm_node][2] != op_packet['db']:
                    print("Error: SF node and data op db mismatch")
                t_node_calls_dict[t_node][idx][1] = op_id
                t_data_ops_dict[op_id] = op_packet
    G = nx.DiGraph()
    G.add_edges_from(traces_dict[tid])
    t_ini_node, trace_depth = find_inode_and_graph_depth(G) # getting initial node
    t_ini_node_type = get_node_type(t_ini_node, node_split_output)
    t_node_calls_dict = remove_self_node_calls(t_node_calls_dict) # Remove self calls in node_calls_dict
    # Get log nodes for this request call graph
    t_logger_nodes = get_logger_nodes_for_request_call_graph(t_node_calls_dict)
    if has_cycle(t_node_calls_dict):
        cycle_ctr += 1
        continue

    trace_packet = {"tid": tid, "node_calls_dict": t_node_calls_dict, "data_ops_dict": t_data_ops_dict,\
                     "initial_node": t_ini_node, "initial_node_type": t_ini_node_type, "logger_nodes": t_logger_nodes}
    if has_cycle(t_node_calls_dict):
        cycle_ctr += 1
    all_trace_packets[tid] = trace_packet
print("Cycle Ctr: ", cycle_ctr)
print("Num Unique nodes: ", len(unique_nodes_check))
save_dict_as_json(all_trace_packets, f'enrichment_runs/{workload_name}/all_trace_packets')
# save_dict_as_json(all_trace_packets, f'enrichment_runs/as_dmix1_2/all_trace_packets') ################# AS EXP

Cycle Ctr:  23113
Num Unique nodes:  0


In [15]:
import json

# Load files
with open('enrichment_runs/dmix1_pg_heavy/node_split_output.json') as f:
    node_split = json.load(f)

with open('enrichment_runs/dmix1_pg_heavy/all_trace_packets.json') as f:
    all_packets = json.load(f)

# Build reverse mapping of nodeID -> expected DB type
expected_role = {}

# Handle SL containers
for sl_type, sl_info in node_split['sl_split'].items():
    for node_id in sl_info['nodes_list']:
        expected_role[node_id] = 'Python'

# Handle SF containers (databases)
for db_type, db_info in node_split['sf_split'].items():
    for node_id in db_info['nodes_list']:
        expected_role[node_id] = db_type

mismatches = set()

for tid, packet in all_packets.items():
    data_ops = packet.get('data_ops_dict', {})
    node_calls = packet.get('node_calls_dict', {})

    for caller_node, calls in node_calls.items():
        for callee_info in calls:
            callee_node = callee_info[0]
            data_op_id = callee_info[1]

            # Determine expected type of callee
            expected = expected_role.get(callee_node)

            if data_op_id != -1:
                op_info = data_ops[str(data_op_id)]
                actual = op_info['db']
            else:
                actual = 'Python'

            if expected != actual:
                mismatches.add(callee_node)
if mismatches:
    print(f"Found {len(mismatches)} mismatches")
    print(mismatches)

Found 77 mismatches
{'n103', 'n5845', 'n5441', 'n7324', 'n8589', 'n5451', 'n2016', 'n7139', 'n5431', 'n8905', 'n3931', 'n4914', 'n6278', 'n2160', 'n4221', 'n2607', 'n7726', 'n6396', 'n4141', 'n8436', 'n4835', 'n7222', 'n7960', 'n384', 'n9367', 'n2267', 'n8408', 'n4815', 'n9136', 'n9555', 'n4576', 'n2048', 'n8177', 'n7885', 'n8838', 'n4132', 'n5278', 'n744', 'n7265', 'n1232', 'n2028', 'n4594', 'n1898', 'n8699', 'n3854', 'n8215', 'n848', 'n1196', 'n1242', 'n1826', 'n1082', 'n8189', 'n8782', 'n6167', 'n141', 'n7209', 'n7186', 'n7447', 'n5223', 'n3501', 'n1553', 'n483', 'n6166', 'n7244', 'n7948', 'n2474', 'n3253', 'n2143', 'n5406', 'n2357', 'n2812', 'n6360', 'n4340', 'n4797', 'n5847', 'n158', 'n8555'}


# Testing


In [ ]:
tid_inodes_cd = {}
traces_dict = pkl_to_dict('traces/exp_544nodes_100ktraces.pkl')
for tid, e_list in traces_dict.items():
    G = nx.DiGraph()
    G.add_edges_from(e_list)
    initial_node, trace_depth = find_inode_and_graph_depth(G)
    if tid not in tid_inodes_cd:
        tid_inodes_cd[tid] = [initial_node, trace_depth]
save_dict_as_json(tid_inodes_cd, f'node_and_trace_details/544_100k_inode_cd_data')

Number of tids:  99991
Number of tids with no sf:  231


In [21]:
# third_elements = [value[2] for value in tdd.values() if len(value) > 2]
# next(iter(atp.items()))
def get_all_logger_nodes(data):
    logger_nodes = []
    # Iterate over the dictionary and extract logger_nodes
    for val in data.values():
        # print(val['logger_nodes'])
        if len(val['logger_nodes']) > 0:
            for ln in val['logger_nodes']:
                logger_nodes.append(ln)
    return logger_nodes
all_logger_nodes = get_all_logger_nodes(atp)
print(set(all_logger_nodes))

{'n7006', 'n812', 'n5971', 'n6076', 'n785', 'n2769', 'n8290', 'n1357', 'n6650', 'n3291', 'n2587', 'n945', 'n6141', 'n7475', 'n7547', 'n6212', 'n7924', 'n5500', 'n1410', 'n6342', 'n1765', 'n3820', 'n7356', 'n5871', 'n2663', 'n1200', 'n1436', 'n6046', 'n4912', 'n2868', 'n4578', 'n4850', 'n728', 'n8132', 'n668', 'n2838', 'n165', 'n2409', 'n7016', 'n8030', 'n895', 'n4800', 'n6255', 'n1914', 'n7642', 'n3392', 'n263', 'n423', 'n341', 'n7200', 'n7361', 'n8307', 'n3826', 'n1220', 'n4558', 'n2453', 'n158', 'n9', 'n1581', 'n7796', 'n4376', 'n3441', 'n7994', 'n7206', 'n5775', 'n7567', 'n5155', 'n6134', 'n4748', 'n1513', 'n1011', 'n2756', 'n3035', 'n4690', 'n2996', 'n7421', 'n2158', 'n6096', 'n6286', 'n2291', 'n7510', 'n995', 'n7549', 'n5510', 'n1439', 'n776', 'n7370', 'n572', 'n3888', 'n1860', 'n7595', 'n1562', 'n5015', 'n2685', 'n5447', 'n8221', 'n5259', 'n4610', 'n4592', 'n2134', 'n6693', 'n4923', 'n6109', 'n7900', 'n942', 'n3827', 'n8167', 'n1964', 'n2826', 'n366', 'n7045', 'n4715', 'n5856', '

In [ ]:
from collections import Counter
traces_dict = pkl_to_dict('traces/exp_497nodes_100ktraces.pkl')
nso_nlist = []
atp_nlist = []
def extract_unique_nodes(data):
    unique_nodes = set()  # Use a set to keep unique values
    
    # Iterate over both sf_split and sl_split keys
    for split_key in ["sf_split", "sl_split"]:
        if split_key in data:
            # Iterate over each nested dictionary
            for service in data[split_key].values():
                # Add nodes_list items to the set
                nodes = service.get("nodes_list", [])
                unique_nodes.update(nodes)
    
    return list(unique_nodes)
nso_nlist = extract_unique_nodes(nso)
l_list = []
for tid in traces_dict:
    edges_list = traces_dict[tid]
    for edge in edges_list:
        l_list.append(edge[0])
        l_list.append(edge[1])
        # if edge[0] not in atp_nlist:
        #     atp_nlist.append(edge[0])
        # if edge[1] not in atp_nlist:
        #     atp_nlist.append(edge[1])

# print(len(set(nso_nlist)))
# print(len(set(atp_nlist)))
# print(set(nso_nlist) == set(atp_nlist))
# print(set(nso_nlist) == set(unique_nodes_check))

top_five = Counter(l_list).most_common(10)
top_10 = [item for item, count in Counter(l_list).most_common(10)]
print(top_five)
print(top_10)

[('n1765', 354503), ('n2134', 210271), ('n4376', 166338), ('n2977', 138532), ('n942', 65478), ('n4202', 56730), ('n5015', 39773), ('n2436', 36547), ('n6952', 36086), ('n6286', 35163)]
['n1765', 'n2134', 'n4376', 'n2977', 'n942', 'n4202', 'n5015', 'n2436', 'n6952', 'n6286']


In [9]:
atp = read_json_file("enrichment_runs/dmix1_pg_heavy/all_trace_packets.json")
node_counter = Counter()

# Iterate over each item in the data dictionary
sync_call_ctr = 0
async_call_ctr = 0
for item in atp.values():
    node_calls_dict = item.get("node_calls_dict", {})
    # Collect the first elements when the second element is not -1
    for calls in node_calls_dict.values():
        for call in calls:
            if call[2] == 1:
                async_call_ctr += 1
            else:
                sync_call_ctr += 1
    first_elements = [
        call[0]
        for calls in node_calls_dict.values()
        # for call in calls
        # if call[1] != -1
    ]
    # Update the counter with the collected elements
    node_counter.update(first_elements)

# Get the top 5 most common first elements
top_5 = node_counter.most_common(20)


# Print the results
print(f"Sync Calls: {sync_call_ctr}")
print(f"Async Calls: {async_call_ctr}")
print("Top 5 most frequent first elements when the second element is not -1:")
for element, count in top_5:
    print(f"Element {element} appears {count} times")

Sync Calls: 874894
Async Calls: 874147
Top 5 most frequent first elements when the second element is not -1:
Element n7590 appears 94186 times
Element n1082 appears 66738 times
Element n744 appears 52571 times
Element n4576 appears 39967 times
Element n2993 appears 36991 times
Element n103 appears 34861 times
Element n7309 appears 32848 times
Element n9555 appears 32632 times
Element n6846 appears 31256 times
Element n2836 appears 30047 times
Element n5451 appears 28640 times
Element n750 appears 21279 times
Element n4835 appears 16607 times
Element n3287 appears 12824 times
Element n4191 appears 12464 times
Element n3675 appears 9843 times
Element n6650 appears 9702 times
Element n4594 appears 8993 times
Element n9365 appears 8163 times
Element n2431 appears 7554 times


In [4]:
'''
GENERATE TID TO LOGGER NODES DICT
'''
exp_name  = "dmix1_pg_heavy"  ## CHANGE FOR EACH RUN
tpkts = "enrichment_runs/{}/all_trace_packets.json".format(exp_name)


with open(tpkts, "r") as f:
    tpkts = json.load(f)

# # print one trace packet
# first_key = next(iter(tpkts))
# tpkt = tpkts[first_key]
# print(tpkt)

# create a dictionary; key is tid and value is list of logger nodes
tid_to_logger_nodes = {}
for tpkt in tpkts.values():
    tid = tpkt["tid"]
    logger_nodes = tpkt["logger_nodes"]
    tid_to_logger_nodes[tid] = logger_nodes
fk = next(iter(tid_to_logger_nodes))
print(tid_to_logger_nodes[fk])
# write the dictionary to a json and write to same dir
with open(f"enrichment_runs/{exp_name}/tid_to_logger_nodes.json", "w") as f:
    json.dump(tid_to_logger_nodes, f)   

['n7019']


In [ ]:
atp = read_json_file("enrichment_runs/dmix1_pg_heavy/all_trace_packets.json")
nso = read_json_file("enrichment_runs/dmix1+pg_heavy/node_split_output.json")

In [ ]:
# Print the first key-value pair of atp
first_key_atp = next(iter(atp))
print(f"First key-value pair of atp: {first_key_atp}: {atp[first_key_atp]}")

# Print the first key-value pair of nso
# first_key_nso = next(iter(nso))
# print(f"First key-value pair of nso: {first_key_nso}: {nso[first_key_nso]}")

# def find_database_type(node_id, node_split_dict):
#     nso = node_split_dict['sf_split']
#     for db_type, db_info in nso.items():
#         # print(db_type, db_info)
#         if node_id in db_info['nodes_list']:
#             return db_type
#     return "Unknown"
# # Check database type for node 'n355'
# node_id = "n355"
# db_type = find_database_type(node_id, nso)
# print(f"Node {node_id} is of type: {db_type}")

First key-value pair of atp: 0b51191915919355084855000e1e53: {'tid': '0b51191915919355084855000e1e53', 'node_calls_dict': {'n2146': [['n7019', -1, 1]], 'n7019': [['n103', 1, 1]]}, 'data_ops_dict': {'1': {'op_id': 1, 'op_type': 'write', 'op_obj_id': 'key_437', 'db': 'Postgres'}}, 'initial_node': 'n2146', 'initial_node_type': 'Python', 'logger_nodes': ['n7019']}


In [19]:
def find_trace_packets(node_id, tracepackets_dict):
    matching_packets = []
    for key, value in tracepackets_dict.items():
        if node_id in value.get('node_calls_dict', {}).keys() or any(node_id in sublist for sublist in sum(value.get('node_calls_dict', {}).values(), [])):
            matching_packets.append((key, value))
        if len(matching_packets) >= 4:
            break
    return matching_packets
matching_packets = find_trace_packets("n355", atp)
for packet in matching_packets:
    print(packet)

('0b524ede15919482083141000f', {'tid': '0b524ede15919482083141000f', 'node_calls_dict': {'n6292': [['n1016', -1, 1]], 'n1016': [['n3611', -1, 0], ['n699', -1, 0], ['n355', 1, 1], ['n1194', 1, 0], ['n5043', -1, 1]]}, 'data_ops_dict': {'1': {'op_id': 1, 'op_type': 'write', 'op_obj_id': 'key_17', 'db': 'MongoDB'}}, 'initial_node': 'n6292', 'initial_node_type': 'Python', 'logger_nodes': ['n1016', 'n699', 'n5043', 'n3611']})
('0b5167e115919488533109000f', {'tid': '0b5167e115919488533109000f', 'node_calls_dict': {'n6292': [['n1016', -1, 1]], 'n1016': [['n5043', -1, 1], ['n355', 2, 0], ['n1194', 2, 1], ['n699', -1, 0], ['n3611', -1, 1]]}, 'data_ops_dict': {'2': {'op_id': 2, 'op_type': 'write', 'op_obj_id': 'key_520', 'db': 'MongoDB'}}, 'initial_node': 'n6292', 'initial_node_type': 'Python', 'logger_nodes': ['n1016', 'n699', 'n5043', 'n3611']})
('0b5167e115919489048428000f', {'tid': '0b5167e115919489048428000f', 'node_calls_dict': {'n6292': [['n1016', -1, 0]], 'n1016': [['n3611', -1, 0], ['n69